## Example of retrieving and applying the colorterm spline values from "The Monster"

In [1]:
import numpy as np
import astropy.units as u
import requests
import yaml

from scipy.interpolate import make_interp_spline

In [2]:
def get_colorterm_spline(colorterm_file_string, band):
    """
    Get the colorterm spline for a specific band.

    This function retrieves the colorterm spline from a specified file.

    Parameters
    ----------
    colorterm_file_string : `str`
        The base string of the colorterm file name.
    band : `str`
        The band for which to get the colorterm spline
        (e.g., 'g', 'r', 'i').

    Returns
    -------
    colorterm_spline : `dict`
        Dict containing the colorterm spline nodes and values, plus
        additional info.
    """
    colorterm_path = "https://raw.githubusercontent.com/lsst-dm/the_monster/refs/heads/main/colorterms"

    colorterm_url = os.path.join(
                colorterm_path,
                colorterm_file_string+f'_{band}.yaml',
            )

    print(f"Fetching {colorterm_url}")
    # Retrieve the file content from the URL
    response = requests.get(colorterm_url, allow_redirects=True)
    # Convert bytes to string
    content = response.content.decode("utf-8")
    assert content != '404: Not Found', f"File {colorterm_url} not found."

    # Load the yaml
    colorterms_dict = yaml.safe_load(content)

    return colorterms_dict

In [3]:
def ab_mag_to_njy(mags):
    """Convert AB magnitudes to flux in nanoJansky
    Parameters
    ----------
    mags : `np.ndarray`
        Array of mags to convert
    Returns
    -------
    fluxes : `np.ndarray`
        Array of fluxes in nJy
    """
    return (mags * u.ABmag).to(u.nJy).value

In [4]:
def apply_colorterms(source_color_flux_1, source_color_flux_2, source_flux, colorterms_dict):
    """Apply the color term spline model.
    Parameters
    ----------
    source_color_flux_1 : `np.ndarray` (N,)
        Array of source fluxes used for color (1).
    source_color_flux_2 : `np.ndarray` (N,)
        Array of source fluxes used for color (2).
    source_flux : `np.ndarray` (N,)
        Array of source fluxes to convert.
    colorterms_dict : `dict`
        Dict containing colorterm spline nodes and values
    Returns
    -------
    model_flux : `np.ndarray` (N,)
        Array of fluxes converted to target system.
    """
    mag_1 = (np.array(source_color_flux_1)*u.nJy).to_value(u.ABmag)
    mag_2 = (np.array(source_color_flux_2)*u.nJy).to_value(u.ABmag)
    mag_color = mag_1 - mag_2
    spl = make_interp_spline(colorterms_dict['nodes'], colorterms_dict['spline_values'])
    model_flux = np.array(source_flux) * np.array(spl(mag_color))
    model_flux -= colorterms_dict['flux_offset']

    # Check that things are in range: colors out of range simply should
    # not be corrected.
    bad = ((mag_color < colorterms_dict['nodes'][0]) |\
           (mag_color > colorterms_dict['nodes'][-1]) |\
           (~np.isfinite(mag_color)))
    model_flux[bad] = np.nan

    return model_flux

### Get the dict of colorterm spline values:

The format of "colorterm_file_string" is "Monster_to_{system}_band", where "system" is the system you want to transform to.

In [5]:
colorterm_file_string = "Monster_to_ComCam_band"
band = "u"

colorterms_dict = get_colorterm_spline(colorterm_file_string, band)

Fetching https://raw.githubusercontent.com/lsst-dm/the_monster/refs/heads/main/colorterms/Monster_to_ComCam_band_u.yaml


In [6]:
colorterms_dict

{'flux_offset': 0.0,
 'nodes': [0.35, 0.4375, 0.5249999999999999, 0.6124999999999999, 0.7],
 'source_color_field_1': 'monster_DES_g_flux',
 'source_color_field_2': 'monster_DES_r_flux',
 'source_field': 'monster_SDSS_u_flux',
 'source_survey': 'Monster',
 'spline_values': [1.1744485105957316,
  1.138572963077543,
  1.1166661995392342,
  1.128442879245455,
  1.144844329061915],
 'target_survey': 'ComCam'}

### Create some dummy mags and test the `apply_colorterms` function:

In [7]:
mag1 = np.asarray([21, 21]) # g
mag2 = np.asarray([20.3,20.6]) # r
mag_source = np.asarray([20, 25])
flux1 = ab_mag_to_njy(mag1)
flux2 = ab_mag_to_njy(mag2)
flux_source = ab_mag_to_njy(mag_source)

In [8]:
print('Input fluxes: ', flux_source)
print('Transformed fluxes: ', apply_colorterms(flux1, flux2, flux_source, colorterms_dict))

Input fluxes:  [36307.80547701   363.07805477]
Transformed fluxes:  [41566.78520104   419.02775079]
